<br>

# Introdução

In [ ]:
import os
import json
import folium
import requests
import numpy as np
import pandas as pd
import geopandas as gpd

In [ ]:
#import shutil
#import urllib.request

In [ ]:
#!pip install traquitanas --upgrade

In [ ]:
import traquitanas.utils as tt

<br>

# Dados Espaciais

Com a estrutura de pastas criada, é possivel fazer o download dos arquivos disponiblizados pelo IBGE. Há uma infinidade de dados e ainda, há a <a title="Link da API" href="http://api.sidra.ibge.gov.br" target="_blank">**_API do SIDRA_**</a> que possibilita obter mais dados.

- https://servicodados.ibge.gov.br/api/docs

In [ ]:
# Inicialmente faz-se necessário criar uma pasta que receberão os dados do IBGE
data_path = os.path.join('..', 'data')
os.makedirs(data_path, exist_ok=True)

In [ ]:
# Define qual o código IBGE do Estado
estado = 35

# Define URL
parameters = {
    'formato': 'application/vnd.geo+json',
    'resolucao': '5',
    'qualidade': '4',
}
url = 'https://servicodados.ibge.gov.br/api/v2/malhas/{}'.format(estado)
r = requests.get(url, params=parameters)
print(r.url)

# Define o nome do arquivo que será salvo as informações do IBGE
geojson_file = os.path.join(data_path, 'divadmin.geojson')

# Save
with open(geojson_file, 'wb') as f:
    f.write(r.content)

In [ ]:
file_encoding = tt.predict_encoding(geojson_file)
file_encoding

In [ ]:
# Map Object
m = folium.Map()

# Folium Object
folium.GeoJson(geojson_file).add_to(m)

# Fit and Plot map
m.fit_bounds(m.get_bounds())
m

<br>

Ou ainda, ao invés de baixar o arquivo, é possivel fazer com o que o mapa seja criado com a leitura dos dados diretamente do site do IBGE. Nessa função o *encoding* já foi definido, evitando o problema mencionado acima.

In [ ]:
r = requests.get(url, params=parameters)
geojson = json.loads(r.text)

In [ ]:
# Map Object
m = folium.Map()

# Folium Object
folium.GeoJson(geojson).add_to(m)

# Fit and Plot map
m.fit_bounds(m.get_bounds())
m

<br>

Uma vez com o mapa na mão, de qualquer que seja o meio que foi obtido, é possivel analisar a "tabela de atributos".
Lá descobrimos que existe o par de coordenadas que define o centroide e, ainda, o 'codarea' que tem o código do IBGE do município.

<br>

# Dados Tabulares

In [ ]:
# Defines IBGE Code State
estado = 35

# Defines URL
url = 'http://servicodados.ibge.gov.br/api/v1/localidades/estados/{}/municipios'.format(estado)
print('{}\n'.format(url))

# Create Dataframe
df = pd.read_json(url)

# Seleciona Colunas
df = df[['id', 'nome']]

# Renomeia Colunas
df = df.rename(columns=lambda x: x.replace('id', 'id_ibge'))

# Adjust dtypes
#df['id_ibge'] = df['id_ibge'].apply(lambda x: str(x))

# Add Random Colum
df['random'] = np.random.uniform(1, 6, df.shape[0])

# Results
print('{}\n'.format(df.dtypes))
display(df.head())

In [ ]:
# Geojson file
geojson_file = os.path.join(data_path, 'divadmin.geojson')

# Geodataframe
gdf = gpd.read_file(geojson_file)

# Adjust dtypes
gdf['codarea'] = pd.to_numeric(gdf['codarea'])

# Drop
gdf.rename({'codarea': 'id_ibge'}, axis=1, inplace=True)
gdf = gdf[['id_ibge', 'geometry']]

# Results
print('{}\n'.format(gdf.dtypes))
display(gdf.head())

In [ ]:
# Create Map
m = folium.Map()

highlight_function = lambda x: {
    'fillColor': '#000000', 
    'color':'#000000',
    'fillOpacity': 30,
    'weight': 1
}

choropleth = folium.Choropleth(
    geo_data=gdf,
    name='choropleth',    
    data=df,
    columns=['id_ibge', 'random', 'nome'],
    key_on='feature.properties.id_ibge',
    fill_color='YlGnBu',
    fill_opacity=0.5,
    line_opacity=0.1,
    legend_name='Legenda',
    smooth_factor=0,
    show=False,
    overlay=True,
    highlight=True,
    highlight_function=highlight_function,
    nan_fill_color='White',
).add_to(m)

# Add Field in geodataframe
choropleth.geojson.add_child(
    folium.features.GeoJsonTooltip(['id_ibge'], labels=False)
)

# Fit and Plot map
m.fit_bounds(m.get_bounds())
m